# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API


6409, Зубов Егор

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [60]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [61]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [62]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [63]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [59]:
device = '/device:GPU:0'

In [64]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()

(64, 10)


c:\Users\ironb\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [65]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.conv1 = tf.keras.layers.Conv2D(channel_1, (5, 5), activation='relu', padding='same')
        self.conv2 = tf.keras.layers.Conv2D(channel_1, (3, 3), activation='relu', padding='same')
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        scores = self.conv1(x)
        scores = self.conv2(scores)
        scores = self.flatten(scores)
        scores = self.fc(scores)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [66]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [26]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):

        
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [27]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.9049882888793945, Accuracy: 7.8125, Val Loss: 2.9482052326202393, Val Accuracy: 10.40000057220459


Iteration 100, Epoch 1, Loss: 2.2200653553009033, Accuracy: 28.3570556640625, Val Loss: 1.9028288125991821, Val Accuracy: 39.099998474121094
Iteration 200, Epoch 1, Loss: 2.0701308250427246, Accuracy: 32.00404357910156, Val Loss: 1.9154263734817505, Val Accuracy: 39.79999923706055
Iteration 300, Epoch 1, Loss: 2.0013835430145264, Accuracy: 33.84032440185547, Val Loss: 1.9118201732635498, Val Accuracy: 36.599998474121094
Iteration 400, Epoch 1, Loss: 1.935369849205017, Accuracy: 35.5946044921875, Val Loss: 1.739432454109192, Val Accuracy: 42.69999694824219
Iteration 500, Epoch 1, Loss: 1.8923349380493164, Accuracy: 36.69847869873047, Val Loss: 1.677945613861084, Val Accuracy: 42.29999923706055
Iteration 600, Epoch 1, Loss: 1.861232876777649, Accuracy: 37.598793029785156, Val Loss: 1.701407790184021, Val Accuracy: 41.0
Iteration 700, Epoch 1, Loss: 1.8358023166656494, Accuracy: 38.291282653808594, Val Loss: 1.6412256956100464, Val Accuracy: 45.69999694824219


Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [28]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.3082430362701416, Accuracy: 6.25, Val Loss: 2.3304615020751953, Val Accuracy: 13.899999618530273
Iteration 100, Epoch 1, Loss: 1.8504542112350464, Accuracy: 34.05012512207031, Val Loss: 1.6003905534744263, Val Accuracy: 44.20000076293945
Iteration 200, Epoch 1, Loss: 1.6887083053588867, Accuracy: 40.376243591308594, Val Loss: 1.4391030073165894, Val Accuracy: 50.099998474121094
Iteration 300, Epoch 1, Loss: 1.6034681797027588, Accuracy: 43.45410919189453, Val Loss: 1.388480305671692, Val Accuracy: 50.599998474121094
Iteration 400, Epoch 1, Loss: 1.5397205352783203, Accuracy: 45.752803802490234, Val Loss: 1.3475120067596436, Val Accuracy: 51.20000076293945
Iteration 500, Epoch 1, Loss: 1.4940361976623535, Accuracy: 47.28044128417969, Val Loss: 1.2966029644012451, Val Accuracy: 54.29999923706055
Iteration 600, Epoch 1, Loss: 1.4658846855163574, Accuracy: 48.28150939941406, Val Loss: 1.2588953971862793, Val Accuracy: 54.70000076293945
Iteration 700, Epoch 1, 

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [29]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.7101120948791504, Accuracy: 10.9375, Val Loss: 2.752746105194092, Val Accuracy: 11.5
Iteration 100, Epoch 1, Loss: 2.234534740447998, Accuracy: 28.016706466674805, Val Loss: 1.8553029298782349, Val Accuracy: 39.70000076293945
Iteration 200, Epoch 1, Loss: 2.0829381942749023, Accuracy: 32.16728973388672, Val Loss: 1.8401802778244019, Val Accuracy: 38.900001525878906
Iteration 300, Epoch 1, Loss: 2.004216432571411, Accuracy: 34.125831604003906, Val Loss: 1.9147692918777466, Val Accuracy: 38.10000228881836
Iteration 400, Epoch 1, Loss: 1.9331066608428955, Accuracy: 36.03881072998047, Val Loss: 1.7354092597961426, Val Accuracy: 43.099998474121094
Iteration 500, Epoch 1, Loss: 1.8883377313613892, Accuracy: 37.12263107299805, Val Loss: 1.6589299440383911, Val Accuracy: 43.599998474121094
Iteration 600, Epoch 1, Loss: 1.855364203453064, Accuracy: 38.134361267089844, Val Loss: 1.6854044198989868, Val Accuracy: 42.69999694824219
Iteration 700, Epoch 1, Loss: 1.8292

Альтернативный менее гибкий способ обучения:

In [30]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)



313/313 [==============================] - 1s 4ms/step - loss: 1.6357 - sparse_categorical_accuracy: 0.4363


[1.6356509923934937, 0.43630000948905945]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [32]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    initializer = tf.initializers.VarianceScaling(scale=2.0,seed=42)
    model = tf.keras.Sequential([
        # Первый сверточный слой
        tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu', input_shape=(32, 32, 3), kernel_initializer = initializer),
        # Второй сверточный слой
        tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu', kernel_initializer = initializer),
        # Разворачиваем данные перед подачей на полносвязный слой
        tf.keras.layers.Flatten(),
        # Полносвязный слой с relu активацией
        tf.keras.layers.Dense(10, activation='softmax', kernel_initializer = initializer),
    ])

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.01434326171875, Accuracy: 20.3125, Val Loss: 4.660521030426025, Val Accuracy: 11.0
Iteration 100, Epoch 1, Loss: 2.0080676078796387, Accuracy: 33.214725494384766, Val Loss: 1.6635068655014038, Val Accuracy: 44.0
Iteration 200, Epoch 1, Loss: 1.8057901859283447, Accuracy: 39.039180755615234, Val Loss: 1.5231361389160156, Val Accuracy: 47.29999923706055
Iteration 300, Epoch 1, Loss: 1.7044486999511719, Accuracy: 41.70992660522461, Val Loss: 1.4389516115188599, Val Accuracy: 49.39999771118164
Iteration 400, Epoch 1, Loss: 1.6254959106445312, Accuracy: 44.05003356933594, Val Loss: 1.3926249742507935, Val Accuracy: 52.10000228881836
Iteration 500, Epoch 1, Loss: 1.5716246366500854, Accuracy: 45.74600601196289, Val Loss: 1.3488250970840454, Val Accuracy: 53.29999923706055
Iteration 600, Epoch 1, Loss: 1.5345008373260498, Accuracy: 46.973793029785156, Val Loss: 1.3216633796691895, Val Accuracy: 54.20000076293945
Iteration 700, Epoch 1, Loss: 1.5025739669799805, A

In [33]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)


313/313 [==============================] - 1s 2ms/step - loss: 1.3853 - sparse_categorical_accuracy: 0.5028


[1.3852585554122925, 0.5027999877929688]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [34]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


c:\Users\ironb\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [35]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.210118293762207, Accuracy: 9.375, Val Loss: 3.173954725265503, Val Accuracy: 12.300000190734863
Iteration 100, Epoch 1, Loss: 2.2484664916992188, Accuracy: 27.165842056274414, Val Loss: 1.8758461475372314, Val Accuracy: 38.70000076293945
Iteration 200, Epoch 1, Loss: 2.0846662521362305, Accuracy: 31.436569213867188, Val Loss: 1.8348736763000488, Val Accuracy: 39.29999923706055
Iteration 300, Epoch 1, Loss: 2.004213571548462, Accuracy: 33.68978500366211, Val Loss: 1.8951705694198608, Val Accuracy: 36.0
Iteration 400, Epoch 1, Loss: 1.9366141557693481, Accuracy: 35.67643356323242, Val Loss: 1.7210983037948608, Val Accuracy: 42.099998474121094
Iteration 500, Epoch 1, Loss: 1.8902784585952759, Accuracy: 36.79203796386719, Val Loss: 1.650027871131897, Val Accuracy: 44.80000305175781
Iteration 600, Epoch 1, Loss: 1.8593567609786987, Accuracy: 37.770381927490234, Val Loss: 1.6862709522247314, Val Accuracy: 41.900001525878906
Iteration 700, Epoch 1, Loss: 1.833640

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [57]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        channel_1, channel_2, num_classes_relu, num_classes_softmax = 32, 64, 100, 10
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Сверточный слой 1: 32 фильтра, MaxPool
        self.conv1 = tf.keras.layers.Conv2D(channel_1, (3, 3), activation='relu', padding='same', kernel_initializer=initializer)
        self.mp1 = tf.keras.layers.MaxPooling2D((2, 2))

        # Сверточный слой 2: 64 фильтра, MaxPool
        self.conv2 = tf.keras.layers.Conv2D(channel_2, (3, 3), activation='relu', padding='same', kernel_initializer=initializer)
        self.mp2 = tf.keras.layers.MaxPooling2D((2, 2))
        self.d1 = tf.keras.layers.Dropout(0.25)

        # Используем flatten для сглаживания и Dropout для того, чтобы избежать переобучения
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(num_classes_relu, activation='relu')
        self.d2 = tf.keras.layers.Dropout(0.25)
        self.fc2 = tf.keras.layers.Dense(num_classes_softmax, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)
        x = self.mp1(x)
        x = self.conv2(x)
        x = self.mp2(x)
        x = self.d1(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.d2(x)
        x = self.fc2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 4.010939598083496, Accuracy: 7.8125, Val Loss: 3.6961634159088135, Val Accuracy: 8.0
Iteration 700, Epoch 1, Loss: 1.594498634338379, Accuracy: 42.77371597290039, Val Loss: 1.2068769931793213, Val Accuracy: 60.10000228881836
Iteration 1400, Epoch 2, Loss: 1.2144452333450317, Accuracy: 56.93159866333008, Val Loss: 1.030329704284668, Val Accuracy: 64.5
Iteration 2100, Epoch 3, Loss: 1.075059175491333, Accuracy: 61.895870208740234, Val Loss: 0.9764723181724548, Val Accuracy: 65.80000305175781
Iteration 2800, Epoch 4, Loss: 0.9817975759506226, Accuracy: 65.3112564086914, Val Loss: 0.9333184361457825, Val Accuracy: 67.5999984741211
Iteration 3500, Epoch 5, Loss: 0.9173476696014404, Accuracy: 67.29833984375, Val Loss: 0.8683657050132751, Val Accuracy: 70.5
Iteration 4200, Epoch 6, Loss: 0.8699458241462708, Accuracy: 69.39437103271484, Val Loss: 0.8932826519012451, Val Accuracy: 70.5
Iteration 4900, Epoch 7, Loss: 0.8323455452919006, Accuracy: 70.06147766113281, Va

Опишите все эксперименты, результаты. Сделайте выводы.

В результате экспериментов с архитектурой нейронных сетей можно сделать следующие выводы:
- Использование методов регуляризации (Dropout) положительно повлияло на конечную точность модели
- Увеличение количества линейных слоев также повлияло на конечную точность модели
- На конечную точность модели также повлияло увеличение количества фильтров на сверточных слоях
- Увеличение количества сверточных слоев может привести к переобучению